In [31]:
# common imports
import numpy as np
import pandas as pd
import joblib, glob
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# machine learning imports


# display setup
pd.set_option("display.max_columns", None)  # the None parameter displays unlimited columns
plt.style.use('seaborn')  # for plots

# 1. Getting the Data

> Each participant has 3 csv files, one for each alignment state.
>
> Taking a look at one file for each state should help gain insight to which dataframe manipulations are necessary
> before inserting it into a machine learning algorithm.

In [148]:
# read right hand file
handright = pd.read_csv(r"extraFiles/HandRight.csv")
alone = pd.read_csv(r"extraFiles/Training/Yael Hagai/Yael636771061659067649Alone.csv")
spontan = pd.read_csv(r"extraFiles/Training/Yael Hagai/Yael636771059544754823Spontan.csv")
sync = pd.read_csv(r"extraFiles/Training/Yael Hagai/Yael636771060699529085Sync.csv")

In [149]:
handright.head()

,Time,Frame ID,Hand Type,# hands,Position X,Position Y,Position Z,Velocity X,Velocity Y,Velocity Z,Pitch,Roll,Yaw,Wrist Pos X,Wrist Pos Y,Wrist Pos Z,Elbow pos X,Elbow Pos Y,Elbow Pos Z,Grab Strenth,Grab Angle,Pinch Strength
0,128.2883,15478,right,1,15.67167,226.2064,20.26470,34.26665,-195.214700,-35.043020,0.270987,-0.017635,-0.277800,46.88831,207.3548,86.23589,203.8337,49.33507,237.4553,0.0,0.206389,0.0
1,128.3048,15480,right,1,16.24119,222.9389,20.11781,36.16443,-180.093900,-4.052799,0.245085,-0.014770,-0.277763,47.54861,205.6568,86.42250,203.9553,47.88272,238.4482,0.0,0.243050,0.0
2,128.3218,15482,right,1,16.72461,220.9709,19.94972,26.35298,-95.088190,-11.157620,0.230796,-0.008779,-0.278792,48.15030,204.5182,86.35149,204.6231,45.35921,236.8544,0.0,0.281134,0.0
3,128.3384,15484,right,1,17.09016,219.7124,19.67679,19.46804,-61.755180,-21.303780,0.217681,-0.010759,-0.279768,48.64175,203.9989,86.16056,204.9939,44.18692,236.0927,0.0,0.277691,0.0
4,128.3551,15486,right,1,17.46017,219.3652,19.13128,21.94964,-8.355943,-35.081790,0.220696,-0.017380,-0.278629,48.93135,203.3777,85.59017,204.5952,42.68003,235.2896,0.0,0.283008,0.0


In [150]:
handright.shape

(4812, 22)

In [151]:
handright.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4812 entries, 0 to 4811
Data columns (total 22 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Time             4812 non-null   float64
 1    Frame ID        4812 non-null   int64  
 2    Hand Type       4812 non-null   object 
 3    # hands         4812 non-null   int64  
 4    Position X      4812 non-null   float64
 5    Position Y      4812 non-null   float64
 6    Position Z      4812 non-null   float64
 7    Velocity X      4812 non-null   float64
 8    Velocity Y      4812 non-null   float64
 9    Velocity Z      4812 non-null   float64
 10   Pitch           4812 non-null   float64
 11   Roll            4812 non-null   float64
 12   Yaw             4812 non-null   float64
 13   Wrist Pos X     4428 non-null   float64
 14   Wrist Pos Y     4428 non-null   float64
 15   Wrist Pos Z     4428 non-null   float64
 16   Elbow pos X     4428 non-null   float64
 17   Elbow Pos Y  

In [152]:
handright.isna().sum()

Time                 0
 Frame ID            0
 Hand Type           0
 # hands             0
 Position X          0
 Position Y          0
 Position Z          0
 Velocity X          0
 Velocity Y          0
 Velocity Z          0
 Pitch               0
 Roll                0
 Yaw                 0
 Wrist Pos X       384
 Wrist Pos Y       384
 Wrist Pos Z       384
 Elbow pos X       384
 Elbow Pos Y       384
 Elbow Pos Z       384
 Grab Strenth        0
 Grab Angle          0
 Pinch Strength      0
dtype: int64

In [153]:
handright[" # hands"].value_counts()

1    4732
2      80
Name:  # hands, dtype: int64

In [154]:
handright[" Hand Type"].value_counts()

right    4738
left       74
Name:  Hand Type, dtype: int64

In [98]:
alone.head()

,Time,Frame ID,Hand Type,# hands,Position X,Position Y,Position Z,Velocity X,Velocity Y,Velocity Z,Pitch,Roll,Yaw,Wrist Pos X,Wrist Pos Y,Wrist Pos Z,Elbow pos X,Elbow Pos Y,Elbow Pos Z,Grab Strenth,Grab Angle,Pinch Strength
0,386.9574,45134,left,1,-158.6170,287.6174,-24.87604,-31.29900,356.5704,107.26580,1.514668,1.042867,1.465832,-196.7802,229.1945,-21.44019,-277.7189,10.56576,67.69565,0.0,0.336463,0.368310
1,386.9746,45136,left,1,-159.2629,294.0763,-23.02113,-69.48521,332.7127,105.50880,1.523402,1.051153,1.481661,-197.4044,235.5681,-20.25847,-280.9406,16.27876,64.76319,0.0,0.399600,0.451096
2,386.9904,45138,left,1,-159.8529,299.8839,-21.48677,-59.82772,370.6227,90.46810,1.520507,1.042137,1.477824,-198.4258,241.6253,-18.72481,-284.1040,21.73676,62.53461,0.0,0.389698,0.464458
3,387.0072,45140,left,1,-160.8173,306.4102,-19.99327,-118.96180,377.6304,95.47686,1.509962,1.042556,1.457889,-199.1599,247.9668,-17.27442,-286.7310,27.91961,61.49978,0.0,0.991447,0.660603
4,387.0237,45142,left,1,-162.4503,313.8571,-18.74539,53.77554,370.3851,90.83872,1.510248,1.046471,1.457001,-200.6748,255.2839,-16.04187,-290.6844,35.04873,59.38720,1.0,2.261015,0.793195


In [99]:
alone.shape

(4537, 22)

In [100]:
alone.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4537 entries, 0 to 4536
Data columns (total 22 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Time             4537 non-null   float64
 1    Frame ID        4537 non-null   int64  
 2    Hand Type       4537 non-null   object 
 3    # hands         4537 non-null   int64  
 4    Position X      4537 non-null   float64
 5    Position Y      4537 non-null   float64
 6    Position Z      4537 non-null   float64
 7    Velocity X      4537 non-null   float64
 8    Velocity Y      4537 non-null   float64
 9    Velocity Z      4537 non-null   float64
 10   Pitch           4537 non-null   float64
 11   Roll            4537 non-null   float64
 12   Yaw             4537 non-null   float64
 13   Wrist Pos X     4537 non-null   float64
 14   Wrist Pos Y     4537 non-null   float64
 15   Wrist Pos Z     4537 non-null   float64
 16   Elbow pos X     4537 non-null   float64
 17   Elbow Pos Y  

In [101]:
alone.isna().sum()

Time               0
 Frame ID          0
 Hand Type         0
 # hands           0
 Position X        0
 Position Y        0
 Position Z        0
 Velocity X        0
 Velocity Y        0
 Velocity Z        0
 Pitch             0
 Roll              0
 Yaw               0
 Wrist Pos X       0
 Wrist Pos Y       0
 Wrist Pos Z       0
 Elbow pos X       0
 Elbow Pos Y       0
 Elbow Pos Z       0
 Grab Strenth      0
 Grab Angle        0
 Pinch Strength    0
dtype: int64

In [102]:
alone[" # hands"].value_counts()

1    4435
2     102
Name:  # hands, dtype: int64

In [103]:
alone[" Hand Type"].value_counts()

left     4486
right      51
Name:  Hand Type, dtype: int64

In [104]:
spontan.head()

,Time,Frame ID,Hand Type,# hands,Position X,Position Y,Position Z,Velocity X,Velocity Y,Velocity Z,Pitch,Roll,Yaw,Wrist Pos X,Wrist Pos Y,Wrist Pos Z,Elbow pos X,Elbow Pos Y,Elbow Pos Z,Grab Strenth,Grab Angle,Pinch Strength
0,175.5415,20769,right,2,146.6630,160.2139,-39.06675,-292.87750,-64.44993,86.72254,0.238520,-0.136291,-0.444049,183.7597,147.6642,18.61219,317.4607,64.20674,209.1401,0.0,0.052556,0.000000
1,175.5415,20769,left,2,-175.4261,238.6634,95.82157,-18.25315,-43.46218,-18.86510,1.609791,1.546077,2.799111,-181.1349,170.0918,87.50933,-253.4343,-62.25551,131.4360,0.0,0.907454,0.450613
2,175.5589,20771,right,2,141.7510,158.8064,-36.19062,-270.43750,-82.31941,187.93000,0.272479,-0.109806,-0.552327,183.8038,145.5503,17.72754,317.7048,58.80313,206.6389,0.0,0.043725,0.000000
3,175.5589,20771,left,2,-176.1160,238.6726,95.89357,-62.86600,36.26385,23.44059,1.609235,1.553108,2.964649,-181.2218,170.0441,87.60751,-249.6812,-63.45592,131.5742,0.0,0.926086,0.472969
4,175.5755,20773,right,2,137.8546,158.4656,-32.12399,-208.59670,-10.16733,226.08150,0.308624,-0.081112,-0.695048,185.9733,144.8633,16.12182,321.1910,46.51823,198.2933,0.0,0.105092,0.000000


In [105]:
spontan.shape

(8773, 22)

In [106]:
spontan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8773 entries, 0 to 8772
Data columns (total 22 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Time             8773 non-null   float64
 1    Frame ID        8773 non-null   int64  
 2    Hand Type       8773 non-null   object 
 3    # hands         8773 non-null   int64  
 4    Position X      8773 non-null   float64
 5    Position Y      8773 non-null   float64
 6    Position Z      8773 non-null   float64
 7    Velocity X      8773 non-null   float64
 8    Velocity Y      8773 non-null   float64
 9    Velocity Z      8773 non-null   float64
 10   Pitch           8773 non-null   float64
 11   Roll            8773 non-null   float64
 12   Yaw             8773 non-null   float64
 13   Wrist Pos X     8773 non-null   float64
 14   Wrist Pos Y     8773 non-null   float64
 15   Wrist Pos Z     8773 non-null   float64
 16   Elbow pos X     8773 non-null   float64
 17   Elbow Pos Y  

In [107]:
spontan.isna().sum()

Time               0
 Frame ID          0
 Hand Type         0
 # hands           0
 Position X        0
 Position Y        0
 Position Z        0
 Velocity X        0
 Velocity Y        0
 Velocity Z        0
 Pitch             0
 Roll              0
 Yaw               0
 Wrist Pos X       0
 Wrist Pos Y       0
 Wrist Pos Z       0
 Elbow pos X       0
 Elbow Pos Y       0
 Elbow Pos Z       0
 Grab Strenth      0
 Grab Angle        0
 Pinch Strength    0
dtype: int64

In [108]:
spontan[" # hands"].value_counts()

2    8492
1     281
Name:  # hands, dtype: int64

In [109]:
spontan[" Hand Type"].value_counts()

right    4527
left     4246
Name:  Hand Type, dtype: int64

In [110]:
sync.head()

,Time,Frame ID,Hand Type,# hands,Position X,Position Y,Position Z,Velocity X,Velocity Y,Velocity Z,Pitch,Roll,Yaw,Wrist Pos X,Wrist Pos Y,Wrist Pos Z,Elbow pos X,Elbow Pos Y,Elbow Pos Z,Grab Strenth,Grab Angle,Pinch Strength
0,291.0203,34075,right,2,77.93928,205.6924,21.990890,-31.144050,-14.796920,124.86100,1.654635,-1.457181,-2.149903,96.24556,138.3030,12.827740,229.8921,-68.10480,65.34229,0.0,0.0,0.0
1,291.0203,34075,left,2,-112.80230,242.7639,4.668582,5.713323,-61.391260,-21.03796,1.446623,1.405847,0.951937,-131.90310,174.4940,5.223116,-257.7061,-42.89592,16.99501,0.0,0.0,0.0
2,291.0363,34077,right,2,76.83206,203.9094,23.754180,-85.668820,-130.091700,97.13650,1.731422,-1.463139,-2.436445,95.74135,137.5909,10.118030,231.0989,-67.86788,61.96756,0.0,0.0,0.0
3,291.0363,34077,left,2,-113.04160,242.6686,4.462260,-14.886850,-3.294883,-9.39946,1.455516,1.406081,0.982829,-132.17560,174.4122,4.426020,-259.8255,-41.99683,14.22190,0.0,0.0,0.0
4,291.0529,34079,right,2,74.30998,200.5741,25.159250,-148.608600,-171.942100,72.13271,1.791300,-1.455910,-2.516997,94.72282,135.5896,8.222761,233.5819,-67.46499,60.28345,0.0,0.0,0.0


In [111]:
sync.shape

(8829, 22)

In [112]:
sync.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8829 entries, 0 to 8828
Data columns (total 22 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Time             8829 non-null   float64
 1    Frame ID        8829 non-null   int64  
 2    Hand Type       8829 non-null   object 
 3    # hands         8829 non-null   int64  
 4    Position X      8829 non-null   float64
 5    Position Y      8829 non-null   float64
 6    Position Z      8829 non-null   float64
 7    Velocity X      8829 non-null   float64
 8    Velocity Y      8829 non-null   float64
 9    Velocity Z      8829 non-null   float64
 10   Pitch           8829 non-null   float64
 11   Roll            8829 non-null   float64
 12   Yaw             8829 non-null   float64
 13   Wrist Pos X     8829 non-null   float64
 14   Wrist Pos Y     8829 non-null   float64
 15   Wrist Pos Z     8829 non-null   float64
 16   Elbow pos X     8829 non-null   float64
 17   Elbow Pos Y  

In [113]:
sync.isna().sum()

Time               0
 Frame ID          0
 Hand Type         0
 # hands           0
 Position X        0
 Position Y        0
 Position Z        0
 Velocity X        0
 Velocity Y        0
 Velocity Z        0
 Pitch             0
 Roll              0
 Yaw               0
 Wrist Pos X       0
 Wrist Pos Y       0
 Wrist Pos Z       0
 Elbow pos X       0
 Elbow Pos Y       0
 Elbow Pos Z       0
 Grab Strenth      0
 Grab Angle        0
 Pinch Strength    0
dtype: int64

In [114]:
sync[" # hands"].value_counts()

2    8604
1     225
Name:  # hands, dtype: int64

In [115]:
sync[" Hand Type"].value_counts()

right    4527
left     4302
Name:  Hand Type, dtype: int64

Clearly just by looking at a few files the data needs to be cleaned.

* Alone has instances with 2 hands and right hand in hand type that need to be removed. Likewise, spontaneous
and sync have instances with 1 hand and a mismatch between right and left hands.

In [170]:
def remove_first7(df):
    return df.drop(df[df["Time"] < df["Time"].min() + 7].index).reset_index(drop=True)

In [207]:
print("Before:", handright.shape)
handright_prepared = handright.drop(handright[(handright[" Hand Type"]=="left") | (handright[" # hands"]==2)].index)
print(handright_prepared.shape)
handright_prepared = remove_first7(handright_prepared) # right hand file without first 7 seconds
print(handright_prepared.shape)
handright_prepared.dropna(inplace=True)
print("After:", handright_prepared.shape)
print("Null values:", handright_prepared.isna().sum().sum())
handright_prepared.head()

Before: (4812, 22)
(4698, 22)
(4277, 22)
After: (3893, 22)
Null values: 0


,Time,Frame ID,Hand Type,# hands,Position X,Position Y,Position Z,Velocity X,Velocity Y,Velocity Z,Pitch,Roll,Yaw,Wrist Pos X,Wrist Pos Y,Wrist Pos Z,Elbow pos X,Elbow Pos Y,Elbow Pos Z,Grab Strenth,Grab Angle,Pinch Strength
0,135.3024,16287,right,1,11.93032,199.9861,4.981665,-35.512210,-165.3466,-57.68153,-0.092033,0.108262,-0.906612,75.45740,206.0521,45.38764,284.3946,140.0167,201.9318,0.0,0.561324,0.0
1,135.3194,16289,right,1,11.19190,195.3075,3.661716,-43.587440,-309.8374,-77.18515,-0.111674,0.126925,-0.904311,74.46141,202.4562,44.28679,282.6278,140.9712,203.6773,0.0,0.626321,0.0
2,135.3357,16291,right,1,10.53308,189.0297,2.829388,-30.403600,-363.0033,-43.79815,-0.140004,0.127219,-0.901862,73.60060,197.4387,43.54082,280.6384,142.5401,206.7526,0.0,0.636596,0.0
3,135.3524,16293,right,1,10.24300,181.9816,2.064330,-15.424000,-423.2120,-43.99948,-0.166623,0.122782,-0.901268,73.19537,191.7202,42.73155,279.7357,148.1382,209.9330,0.0,0.701644,0.0
4,135.3690,16295,right,1,10.02061,174.8592,1.228467,-8.733602,-413.4513,-53.88054,-0.197763,0.127587,-0.897777,72.68671,186.1613,41.98006,278.6890,152.5215,212.1112,0.0,0.786548,0.0


In [208]:
handright.isna().sum()

Time                 0
 Frame ID            0
 Hand Type           0
 # hands             0
 Position X          0
 Position Y          0
 Position Z          0
 Velocity X          0
 Velocity Y          0
 Velocity Z          0
 Pitch               0
 Roll                0
 Yaw                 0
 Wrist Pos X       384
 Wrist Pos Y       384
 Wrist Pos Z       384
 Elbow pos X       384
 Elbow Pos Y       384
 Elbow Pos Z       384
 Grab Strenth        0
 Grab Angle          0
 Pinch Strength      0
dtype: int64

In [201]:
def combine_right(df_alone):
    left = df_alone.drop(df_alone[(df_alone[" Hand Type"]=="right") | (df_alone[" # hands"]==2)].index).reset_index(drop=True)
    left = remove_first7(left)
    left.dropna(inplace=True)
    if len(left) > len(handright_prepared):
        right = handright_prepared[:len(left)].copy()
    else:
        diff = len(handright_prepared) - len(left)
        left = left.drop(left[:diff].index).reset_index(drop=True)
        right = handright_prepared[:len(left)].copy()
    right.Time = left.Time
    right[" Frame ID"] = left[" Frame ID"]
    merged = right.merge(left, how="outer").sort_values("Time").reset_index(drop=True)
    merged[" # hands"] = 2
    return merged

def transform_columns(df):
    # strip removes white spaces in the beginning and end
    # lower changes uppercase letters to lowercase
    df.columns = df.columns.str.strip().str.lower()
    # rename columns without spaces
    df.columns = df.columns.str.replace(" ", "_")
    # change number of hands column name
    df.columns = df.columns.str.replace("#_hands", "n_hands")

    # remove instances with 1 hand detected
    df.drop(df[df["n_hands"] == 1].index, inplace=True)

    # merge right and left hands into one row
    left = df.groupby("hand_type").get_group("left").drop(["hand_type", "n_hands"], axis=1)
    right = df.groupby("hand_type").get_group("right").drop(["hand_type", "n_hands"], axis=1)
    keep_same = {"state", "time", "frame_id"}
    left.columns = left.columns.map(lambda x: x if x in keep_same else x + "_left")
    right.columns = right.columns.map(lambda x: x if x in keep_same else x + "_right")
    merged = left.merge(right, how="outer", on=["state", "time", "frame_id"]).drop("frame_id", axis=1)
    return merged

In [202]:
training_data = []
validation_data = []
training_path = "extraFiles/Training"
training_dirs = glob.glob(training_path + "/*")

for dir in training_dirs:
    training_files = glob.glob(dir + "/*.csv")
    for filename in training_files:
        df = pd.read_csv(filename, index_col=None, header=0)
        if "Alone" in filename:
            df = combine_right(df)
            df.insert(0, "state", 0)
        elif "Sync" in filename:
            df.insert(0, "state", 1)
        elif "Spontan" in filename:
            df.insert(0, "state", 2)
        df = transform_columns(df)
        training_data.append(df)

# frame = pd.concat(training_data, axis=0, ignore_index=True)

In [204]:
training_data[2]

,state,time,position_x_left,position_y_left,position_z_left,velocity_x_left,velocity_y_left,velocity_z_left,pitch_left,roll_left,yaw_left,wrist_pos_x_left,wrist_pos_y_left,wrist_pos_z_left,elbow_pos_x_left,elbow_pos_y_left,elbow_pos_z_left,grab_strenth_left,grab_angle_left,pinch_strength_left,position_x_right,position_y_right,position_z_right,velocity_x_right,velocity_y_right,velocity_z_right,pitch_right,roll_right,yaw_right,wrist_pos_x_right,wrist_pos_y_right,wrist_pos_z_right,elbow_pos_x_right,elbow_pos_y_right,elbow_pos_z_right,grab_strenth_right,grab_angle_right,pinch_strength_right
0,0,525.8185,-52.96039,184.5535,-2.856696,162.15270,127.60520,-85.83782,1.238877,1.021919,1.199751,-101.04550,132.5940,4.449887,-307.7127,-12.177800,5.125422,0.0,0.073635,0.0,11.93032,199.9861,4.981665,-35.512210,-165.3466,-57.68153,-0.092033,0.108262,-0.906612,75.45740,206.0521,45.38764,284.3946,140.0167,201.9318,0.0,0.561324,0.0
1,0,525.8351,-50.04434,185.5024,-4.845364,102.82480,-24.84337,-79.46504,1.244299,1.132981,1.171960,-95.51471,131.5824,2.853924,-313.0367,3.746006,-0.704697,0.0,0.018766,0.0,11.19190,195.3075,3.661716,-43.587440,-309.8374,-77.18515,-0.111674,0.126925,-0.904311,74.46141,202.4562,44.28679,282.6278,140.9712,203.6773,0.0,0.626321,0.0
2,0,525.8516,-49.07749,182.2930,-6.442950,12.12360,-258.75260,-99.40902,1.220772,1.250050,1.152816,-94.32024,128.7403,2.742160,-322.1467,20.402740,-2.617105,0.0,0.018409,0.0,10.53308,189.0297,2.829388,-30.403600,-363.0033,-43.79815,-0.140004,0.127219,-0.901862,73.60060,197.4387,43.54082,280.6384,142.5401,206.7526,0.0,0.636596,0.0
3,0,525.8682,-50.89614,176.7708,-9.005262,-53.57292,-271.01500,-143.03330,1.096965,1.535388,1.114818,-99.72611,127.6519,5.047988,-331.5826,28.883190,-7.521224,0.0,0.149852,0.0,10.24300,181.9816,2.064330,-15.424000,-423.2120,-43.99948,-0.166623,0.122782,-0.901268,73.19537,191.7202,42.73155,279.7357,148.1382,209.9330,0.0,0.701644,0.0
4,0,525.8848,-51.76074,174.1853,-10.600340,-40.95883,-140.44390,-59.85620,1.004597,1.760037,1.091361,-102.66840,128.2257,6.734126,-333.9562,27.848030,-3.312394,0.0,0.242324,0.0,10.02061,174.8592,1.228467,-8.733602,-413.4513,-53.88054,-0.197763,0.127587,-0.897777,72.68671,186.1613,41.98006,278.6890,152.5215,212.1112,0.0,0.786548,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4098,0,593.7177,-44.79453,123.7657,-6.241947,274.44630,106.78920,125.05260,1.234747,0.282478,1.476095,-114.52440,107.5486,-6.354107,-354.6525,41.165710,45.739930,0.0,0.401728,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4099,0,593.7343,-40.77390,126.7342,-3.452266,223.52940,209.72390,160.24900,1.265178,0.283400,1.484591,-110.50080,110.7256,-3.791958,-351.3217,53.684260,55.643090,0.0,0.190267,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4100,0,593.7509,-38.56184,127.6858,-2.086977,254.93310,109.66130,157.34590,1.279585,0.280169,1.489703,-108.34430,111.9989,-2.676469,-349.8930,59.758350,58.190420,0.0,0.042665,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4101,0,593.7681,-35.88303,132.9386,0.526575,185.86870,204.98220,154.04880,1.299061,0.298048,1.489702,-105.43440,116.3043,0.041351,-346.9033,66.470160,63.191740,0.0,0.000000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
r = handright_prepared[:100].copy()
l = alone[:200]
# r.Time = l.Time
# r[" Frame ID"] = l[" Frame ID"]

diff = len(l) - len(r)
diff

# l.drop(l[:diff].index).reset_index(drop=True)
l
# merged = r.merge(l, how="outer").sort_values("Time").reset_index(drop=True)
# merged.groupby("Time")[" # hands"].sum()

In [13]:
def transform_time(df):
    # adds a column with the second
    df["start_time"] = df.time.astype(int) - df.time.min().astype(int)
    start = df.start_time.min() + 7 # remove first 7 seconds
    first_7_seconds = df[df["start_time"] < start].index
    # drop is True to prevent from adding the old indices as a new column
    df_new = df.drop(first_7_seconds).reset_index(drop=True)
    # reset index (drop = False) adds round time as the left column in the DataFrame
    df = df.groupby("start_time").nth([0, 1, 2, 3]).reset_index() # first four instances per second
    df.drop(df[df["start_time"] < df["start_time"].min() + 7].index).reset_index(drop=True)
    df.drop("start_time", axis=1, inplace=True)
    return df

In [14]:
transform_time(XXX)

,state,time,position_x_left,position_y_left,position_z_left,velocity_x_left,velocity_y_left,velocity_z_left,pitch_left,roll_left,yaw_left,wrist_pos_x_left,wrist_pos_y_left,wrist_pos_z_left,elbow_pos_x_left,elbow_pos_y_left,elbow_pos_z_left,grab_strenth_left,grab_angle_left,pinch_strength_left,position_x_right,position_y_right,position_z_right,velocity_x_right,velocity_y_right,velocity_z_right,pitch_right,roll_right,yaw_right,wrist_pos_x_right,wrist_pos_y_right,wrist_pos_z_right,elbow_pos_x_right,elbow_pos_y_right,elbow_pos_z_right,grab_strenth_right,grab_angle_right,pinch_strength_right
0,alone,518.8087,-97.60160,164.1213,42.812550,104.25890,-491.67300,-55.42755,2.939312,0.045867,1.661706,-166.7207,164.8320,27.585470,-378.1662,65.893300,-69.90282,0.0,0.741855,0.000000,8.843476,221.2352,6.470119,104.51340,-281.738800,15.508580,0.100931,-0.002752,-0.921538,73.24500,218.7046,46.08551,278.6815,133.4698,198.1875,0.0,0.358833,0.0
1,alone,518.8252,-95.78062,157.6353,42.180840,148.96830,-351.73860,-36.32422,2.294397,0.074132,1.623222,-165.4907,155.6419,29.710370,-380.3233,52.924410,-55.71872,0.0,0.827916,0.000000,10.148750,216.5558,6.559062,66.57184,-279.229500,6.164072,0.084091,0.006277,-0.915060,74.37665,214.7959,46.51600,281.3379,132.6930,198.2479,0.0,0.368574,0.0
2,alone,518.8418,-94.39360,154.6338,41.931710,161.69580,-349.91170,-29.04277,1.962658,0.086932,1.602898,-164.3658,151.5701,31.050590,-381.7444,48.550320,-47.27909,0.0,0.920401,0.000000,11.245270,213.2907,7.028064,64.02025,-131.828100,36.326210,0.079246,0.007753,-0.913869,75.47279,211.6672,46.97842,283.2765,128.6095,197.0070,0.0,0.308148,0.0
3,alone,518.8587,-92.02553,150.6723,42.101190,117.34750,-187.17810,24.42598,1.545560,0.124819,1.567605,-162.2663,144.6214,33.945770,-383.4688,40.271510,-30.72728,0.0,1.034603,0.000000,12.286960,210.7743,7.346128,55.01450,-144.576300,7.151942,0.055594,0.019090,-0.913344,76.48927,209.9682,47.27846,284.0101,124.1224,196.1059,0.0,0.237091,0.0
4,alone,519.0077,-59.46984,133.8906,44.374630,224.32980,76.83612,14.93058,1.039546,0.386180,1.336244,-126.7760,111.5562,52.403780,-362.4101,32.338940,99.34253,0.0,0.701896,0.000000,15.127780,204.0240,7.815229,-14.56217,-6.400454,-8.558474,-0.011722,0.059693,-0.896614,78.63611,206.0877,48.67214,286.6440,123.2016,198.3545,0.0,0.204624,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299,alone,592.0610,-14.48008,198.6138,21.539440,-27.59156,158.53280,9.32160,1.567686,0.973923,1.566222,-59.8157,143.6770,14.271110,-264.3931,-7.235324,18.94250,0.0,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
300,alone,593.0053,-129.23140,199.5141,-1.808334,-143.93890,146.42720,45.25815,1.539906,0.592593,1.550015,-186.9074,158.7414,-8.840541,-292.6154,-71.992000,-27.97601,1.0,3.072773,0.774364,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
301,alone,593.0224,-131.25110,202.0273,-1.129089,-108.58520,141.61760,32.49025,1.582844,0.607091,1.579173,-188.2238,160.4451,-9.720588,-297.2022,-68.870490,-27.50528,1.0,3.141593,0.778001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
302,alone,593.0385,-132.50170,203.9422,-0.623775,-92.13113,130.00810,30.82561,1.617976,0.624429,1.604965,-188.8173,161.5929,-10.415820,-303.9758,-64.893780,-25.26640,1.0,3.141593,0.760941,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


alone["Time"] = alone["Time"] - alone["Time"].min()

    # reset index (drop = False) adds round time as the left column in the DataFrame
    df = df.groupby("start_time").nth([0, 1, 2, 3]).reset_index() # first four instances per second

In [121]:
alone

,Time,Frame ID,Hand Type,# hands,Position X,Position Y,Position Z,Velocity X,Velocity Y,Velocity Z,Pitch,Roll,Yaw,Wrist Pos X,Wrist Pos Y,Wrist Pos Z,Elbow pos X,Elbow Pos Y,Elbow Pos Z,Grab Strenth,Grab Angle,Pinch Strength
0,518.8087,149620,left,1,-97.60160,164.1213,42.812550,104.2589,-491.6730,-55.42755,2.939312,0.045867,1.661706,-166.7207,164.8320,27.585470,-378.1662,65.89330,-69.90282,0.0,0.741855,0.0
1,518.8252,149622,left,1,-95.78062,157.6353,42.180840,148.9683,-351.7386,-36.32422,2.294397,0.074132,1.623222,-165.4907,155.6419,29.710370,-380.3233,52.92441,-55.71872,0.0,0.827916,0.0
2,518.8418,149623,left,1,-94.39360,154.6338,41.931710,161.6958,-349.9117,-29.04277,1.962658,0.086932,1.602898,-164.3658,151.5701,31.050590,-381.7444,48.55032,-47.27909,0.0,0.920401,0.0
3,518.8587,149625,left,1,-92.02553,150.6723,42.101190,117.3475,-187.1781,24.42598,1.545560,0.124819,1.567605,-162.2663,144.6214,33.945770,-383.4688,40.27151,-30.72728,0.0,1.034603,0.0
4,518.8749,149627,left,1,-89.31953,148.1826,42.670650,155.3344,-135.6337,35.87055,1.377898,0.167855,1.535920,-159.4586,138.6087,36.681950,-381.0906,29.07133,-17.01065,0.0,1.166705,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4521,593.7177,158254,left,1,-44.79453,123.7657,-6.241947,274.4463,106.7892,125.05260,1.234747,0.282478,1.476095,-114.5244,107.5486,-6.354107,-354.6525,41.16571,45.73993,0.0,0.401728,0.0
4522,593.7343,158256,left,1,-40.77390,126.7342,-3.452266,223.5294,209.7239,160.24900,1.265178,0.283400,1.484591,-110.5008,110.7256,-3.791958,-351.3217,53.68426,55.64309,0.0,0.190267,0.0
4523,593.7509,158257,left,1,-38.56184,127.6858,-2.086977,254.9331,109.6613,157.34590,1.279585,0.280169,1.489703,-108.3443,111.9989,-2.676469,-349.8930,59.75835,58.19042,0.0,0.042665,0.0
4524,593.7681,158259,left,1,-35.88303,132.9386,0.526575,185.8687,204.9822,154.04880,1.299061,0.298048,1.489702,-105.4344,116.3043,0.041351,-346.9033,66.47016,63.19174,0.0,0.000000,0.0


In [19]:
handright["start_time"] = handright.Time.astype(int) - handright.Time.min().astype(int)

In [20]:
handright

,Time,Frame ID,Hand Type,# hands,Position X,Position Y,Position Z,Velocity X,Velocity Y,Velocity Z,Pitch,Roll,Yaw,Wrist Pos X,Wrist Pos Y,Wrist Pos Z,Elbow pos X,Elbow Pos Y,Elbow Pos Z,Grab Strenth,Grab Angle,Pinch Strength,start_time
0,135.0024,16253,right,1,8.843476,221.2352,6.470119,104.51340,-281.7388,15.508580,0.100931,-0.002752,-0.921538,73.24500,218.7046,46.08551,278.6815,133.4698,198.1875,0.0,0.358833,0.0,0
1,135.0197,16255,right,1,10.148750,216.5558,6.559062,66.57184,-279.2295,6.164072,0.084091,0.006277,-0.915060,74.37665,214.7959,46.51600,281.3379,132.6930,198.2479,0.0,0.368574,0.0,0
2,135.0359,16257,right,1,11.245270,213.2907,7.028064,64.02025,-131.8281,36.326210,0.079246,0.007753,-0.913869,75.47279,211.6672,46.97842,283.2765,128.6095,197.0070,0.0,0.308148,0.0,0
3,135.0526,16259,right,1,12.286960,210.7743,7.346128,55.01450,-144.5763,7.151942,0.055594,0.019090,-0.913344,76.48927,209.9682,47.27846,284.0101,124.1224,196.1059,0.0,0.237091,0.0,0
4,135.0695,16260,right,1,12.752960,209.5283,7.357905,53.77913,-143.7969,1.359107,0.045028,0.027953,-0.911791,76.88019,209.1200,47.37424,284.2625,121.8424,195.5516,0.0,0.217962,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4290,208.1942,24687,right,1,-15.404800,208.0276,9.980799,167.86240,510.0852,-105.529500,0.235165,-0.401107,-0.282299,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.503490,0.0,73
4291,208.2108,24689,right,1,-12.678650,218.1196,9.314082,145.00970,562.3594,-20.707010,0.309213,-0.548390,-0.263492,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.403687,0.0,73
4292,208.2276,24691,right,1,-9.643562,228.2807,9.722980,180.42160,591.9019,28.880570,0.369046,-0.704982,-0.234337,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.310940,0.0,73
4293,208.2443,24692,right,1,-7.980715,233.5056,10.419080,192.45930,604.7344,80.566980,0.381206,-0.770612,-0.222084,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.286526,0.0,73


In [21]:
handright[handright["start_time"] > handright.start_time.min() + 6]

,Time,Frame ID,Hand Type,# hands,Position X,Position Y,Position Z,Velocity X,Velocity Y,Velocity Z,Pitch,Roll,Yaw,Wrist Pos X,Wrist Pos Y,Wrist Pos Z,Elbow pos X,Elbow Pos Y,Elbow Pos Z,Grab Strenth,Grab Angle,Pinch Strength,start_time
420,142.0000,17060,right,1,14.658960,187.5836,53.395480,36.19220,60.84531,-22.857380,0.617264,0.204835,-0.196015,37.08945,151.1525,117.2716,164.8685,48.20160,334.3133,0.0,2.649905,0.620808,7
421,142.0166,17062,right,1,15.622050,189.8192,53.152170,57.94945,153.91340,-15.265450,0.626890,0.197027,-0.187188,37.69561,152.7267,116.7648,168.3619,50.00497,332.1900,0.0,2.654114,0.628246,7
422,142.0335,17064,right,1,17.333340,192.7994,53.030010,104.04750,165.17100,-6.662246,0.639270,0.216243,-0.178939,38.83024,154.8900,116.3512,169.6885,49.15894,330.1981,0.0,2.626608,0.608577,7
423,142.0501,17065,right,1,18.304800,194.3095,52.912470,111.88120,173.91010,-13.536420,0.641245,0.228430,-0.173522,39.47743,156.2263,116.2323,170.8151,48.90745,328.9917,0.0,2.635067,0.606528,7
424,142.0667,17067,right,1,20.644430,197.9036,52.539080,148.56490,215.47750,-19.559410,0.655173,0.232834,-0.152595,40.67534,158.5628,115.4412,172.6581,43.25099,323.5691,0.0,2.684964,0.606563,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4290,208.1942,24687,right,1,-15.404800,208.0276,9.980799,167.86240,510.08520,-105.529500,0.235165,-0.401107,-0.282299,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.503490,0.000000,73
4291,208.2108,24689,right,1,-12.678650,218.1196,9.314082,145.00970,562.35940,-20.707010,0.309213,-0.548390,-0.263492,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.403687,0.000000,73
4292,208.2276,24691,right,1,-9.643562,228.2807,9.722980,180.42160,591.90190,28.880570,0.369046,-0.704982,-0.234337,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.310940,0.000000,73
4293,208.2443,24692,right,1,-7.980715,233.5056,10.419080,192.45930,604.73440,80.566980,0.381206,-0.770612,-0.222084,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.286526,0.000000,73


In [22]:
sync.head()

,Time,Frame ID,Hand Type,# hands,Position X,Position Y,Position Z,Velocity X,Velocity Y,Velocity Z,Pitch,Roll,Yaw,Wrist Pos X,Wrist Pos Y,Wrist Pos Z,Elbow pos X,Elbow Pos Y,Elbow Pos Z,Grab Strenth,Grab Angle,Pinch Strength
0,427.2310,139071,right,2,69.65445,209.9525,-18.54797,-65.04708,338.2631,-102.57630,0.865190,-1.009409,-0.763543,112.1363,162.7645,10.928120,275.1779,-7.356547,87.30630,0.0,0.059833,0.0
1,427.2310,139071,left,2,-87.18215,204.5752,-40.50642,171.17460,247.9453,-96.48488,1.239694,0.685550,1.220096,-137.3886,158.1433,-27.829490,-261.8767,-47.873840,30.62584,0.0,0.000000,0.0
2,427.2483,139073,right,2,68.97833,215.1191,-20.12599,-44.44045,302.9170,-83.93305,0.863391,-1.005569,-0.766907,111.5590,167.9522,9.239409,272.0606,-5.603438,83.23297,0.0,0.090605,0.0
3,427.2483,139073,left,2,-85.20093,209.7387,-41.18375,70.93041,344.8777,-46.40427,1.294470,0.710267,1.276068,-135.5462,162.8801,-30.982930,-259.9806,-43.469070,26.40578,0.0,0.000000,0.0
4,427.2641,139075,right,2,68.11068,220.9579,-21.77186,-22.08052,307.0574,-67.08289,0.875840,-1.017380,-0.767062,110.1764,173.1205,7.244619,267.7276,-2.693964,82.23973,0.0,0.069295,0.0


In [23]:
sync.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9052 entries, 0 to 9051
Data columns (total 22 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Time             9052 non-null   float64
 1    Frame ID        9052 non-null   int64  
 2    Hand Type       9052 non-null   object 
 3    # hands         9052 non-null   int64  
 4    Position X      9052 non-null   float64
 5    Position Y      9052 non-null   float64
 6    Position Z      9052 non-null   float64
 7    Velocity X      9052 non-null   float64
 8    Velocity Y      9052 non-null   float64
 9    Velocity Z      9052 non-null   float64
 10   Pitch           9052 non-null   float64
 11   Roll            9052 non-null   float64
 12   Yaw             9052 non-null   float64
 13   Wrist Pos X     9052 non-null   float64
 14   Wrist Pos Y     9052 non-null   float64
 15   Wrist Pos Z     9052 non-null   float64
 16   Elbow pos X     9052 non-null   float64
 17   Elbow Pos Y  

In [24]:
# strip removes whitespaces, lower changes the uppercase letters to lowercase
sync.columns = sync.columns.str.strip().str.lower()
sync.columns

Index(['time', 'frame id', 'hand type', '# hands', 'position x', 'position y',
       'position z', 'velocity x', 'velocity y', 'velocity z', 'pitch', 'roll',
       'yaw', 'wrist pos x', 'wrist pos y', 'wrist pos z', 'elbow pos x',
       'elbow pos y', 'elbow pos z', 'grab strenth', 'grab angle',
       'pinch strength'],
      dtype='object')

In [25]:
# rename columns without spaces
sync.columns = sync.columns.str.replace(" ", "_")
# change number of hands column name
sync.columns = sync.columns.str.replace("#_hands", "n_hands")

In [26]:
sync.n_hands.unique()

array([2], dtype=int64)

In [27]:
index_numbers = sync[sync["n_hands"] == 1].index

In [28]:
sync.drop(index_numbers, inplace=False)

,time,frame_id,hand_type,n_hands,position_x,position_y,position_z,velocity_x,velocity_y,velocity_z,pitch,roll,yaw,wrist_pos_x,wrist_pos_y,wrist_pos_z,elbow_pos_x,elbow_pos_y,elbow_pos_z,grab_strenth,grab_angle,pinch_strength
0,427.2310,139071,right,2,69.65445,209.9525,-18.54797,-65.047080,338.263100,-102.57630,0.865190,-1.009409,-0.763543,112.1363,162.7645,10.928120,275.1779,-7.356547,87.30630,0.0,0.059833,0.0
1,427.2310,139071,left,2,-87.18215,204.5752,-40.50642,171.174600,247.945300,-96.48488,1.239694,0.685550,1.220096,-137.3886,158.1433,-27.829490,-261.8767,-47.873840,30.62584,0.0,0.000000,0.0
2,427.2483,139073,right,2,68.97833,215.1191,-20.12599,-44.440450,302.917000,-83.93305,0.863391,-1.005569,-0.766907,111.5590,167.9522,9.239409,272.0606,-5.603438,83.23297,0.0,0.090605,0.0
3,427.2483,139073,left,2,-85.20093,209.7387,-41.18375,70.930410,344.877700,-46.40427,1.294470,0.710267,1.276068,-135.5462,162.8801,-30.982930,-259.9806,-43.469070,26.40578,0.0,0.000000,0.0
4,427.2641,139075,right,2,68.11068,220.9579,-21.77186,-22.080520,307.057400,-67.08289,0.875840,-1.017380,-0.767062,110.1764,173.1205,7.244619,267.7276,-2.693964,82.23973,0.0,0.069295,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9047,502.1665,147702,left,2,-93.68062,253.3679,50.37835,32.269880,-161.195300,-137.36660,1.246310,1.297078,0.956325,-126.9241,191.1363,58.912840,-302.6545,12.995740,26.75648,0.0,0.000000,0.0
9048,502.1830,147704,right,2,108.90170,178.8129,31.02029,-15.531000,-5.445981,-50.80544,0.355584,-2.007045,-0.671661,153.5864,150.0203,78.108750,363.4342,50.553340,176.69620,0.0,0.144381,0.0
9049,502.1830,147704,left,2,-93.12487,250.4072,48.05837,32.141470,-157.050300,-124.67950,1.208146,1.327065,0.901075,-126.4018,188.6597,59.136610,-308.9988,15.970830,37.08796,0.0,0.000000,0.0
9050,502.1996,147706,right,2,108.96690,179.1278,30.35721,5.587708,20.259880,-34.82480,0.356135,-2.005629,-0.669985,153.5919,150.3768,77.516830,364.0365,52.970890,176.88680,0.0,0.148039,0.0


In [29]:
sync.time = sync.time.round(2)

In [30]:
sync.groupby(["time", "hand_type"]).mean()

frame_id  n_hands  position_x  position_y  position_z  \
time   hand_type                                                          
427.23 left       139071.0      2.0   -87.18215    204.5752   -40.50642   
       right      139071.0      2.0    69.65445    209.9525   -18.54797   
427.25 left       139073.0      2.0   -85.20093    209.7387   -41.18375   
       right      139073.0      2.0    68.97833    215.1191   -20.12599   
427.26 left       139075.0      2.0   -83.49533    214.9115   -42.59961   
...                    ...      ...         ...         ...         ...   
502.17 right      147702.0      2.0   109.17140    179.3461    31.84416   
502.18 left       147704.0      2.0   -93.12487    250.4072    48.05837   
       right      147704.0      2.0   108.90170    178.8129    31.02029   
502.20 left       147706.0      2.0   -92.35281    246.6158    45.84472   
       right      147706.0      2.0   108.96690    179.1278    30.35721   

                  velocity_x  velocity_y  velocity_z     pitch      roll  \
time   hand_type                                                           
427.23 left       171.174600  247.945300   -96.48488  1.239694  0.685550   
       right      -65.047080  338.263100  -102.57630  0.865190 -1.009409   
427.25 left        70.930410  344.877700   -46.40427  1.294470  0.710267   
       right      -44.440450  302.917000   -83.93305  0.863391 -1.005569   
427.26 left        85.710340  348.234600  -102.02660  1.331296  0.730335   
...                      ...         ...         ...       ...       ...   
502.17 right      -46.123690  -69.914710   -57.69822  0.355285 -1.995915   
502.18 left        32.141470 -157.050300  -124.67950  1.208146  1.327065   
       right      -15.531000   -5.445981   -50.80544  0.355584 -2.007045   
502.20 left        56.623870 -228.451000  -134.96170  1.180058  1.358709   
       right        5.587708   20.259880   -34.82480  0.356135 -2.005629   

                       yaw  wrist_pos_x  wrist_pos_y  wrist_pos_z  \
time   hand_type                                                    
427.23 left       1.220096    -137.3886     158.1433   -27.829490   
       right     -0.763543     112.1363     162.7645    10.928120   
427.25 left       1.276068    -135.5462     162.8801   -30.982930   
       right     -0.766907     111.5590     167.9522     9.239409   
427.26 left       1.312464    -133.7321     167.7086   -33.756650   
...                    ...          ...          ...          ...   
502.17 right     -0.671905     153.8897     150.4046    78.829510   
502.18 left       0.901075    -126.4018     188.6597    59.136610   
       right     -0.671661     153.5864     150.0203    78.108750   
502.20 left       0.855786    -125.3931     185.2068    58.869600   
       right     -0.669985     153.5919     150.3768    77.516830   

                  elbow_pos_x  elbow_pos_y  elbow_pos_z  grab_strenth  \
time   hand_type                                                        
427.23 left         -261.8767   -47.873840     30.62584           0.0   
       right         275.1779    -7.356547     87.30630           0.0   
427.25 left         -259.9806   -43.469070     26.40578           0.0   
       right         272.0606    -5.603438     83.23297           0.0   
427.26 left         -257.4434   -37.249580     29.84401           0.0   
...                       ...          ...          ...           ...   
502.17 right         362.5193    46.762530    175.68630           0.0   
502.18 left         -308.9988    15.970830     37.08796           0.0   
       right         363.4342    50.553340    176.69620           0.0   
502.20 left         -314.7061    19.054550     44.61267           0.0   
       right         364.0365    52.970890    176.88680           0.0   

                  grab_angle  pinch_strength  
time   hand_type                              
427.23 left         0.000000             0.0  
       right        0.059833             0.0  
427.25 left      

In [31]:
sync.head()

,time,frame_id,hand_type,n_hands,position_x,position_y,position_z,velocity_x,velocity_y,velocity_z,pitch,roll,yaw,wrist_pos_x,wrist_pos_y,wrist_pos_z,elbow_pos_x,elbow_pos_y,elbow_pos_z,grab_strenth,grab_angle,pinch_strength
0,427.23,139071,right,2,69.65445,209.9525,-18.54797,-65.04708,338.2631,-102.57630,0.865190,-1.009409,-0.763543,112.1363,162.7645,10.928120,275.1779,-7.356547,87.30630,0.0,0.059833,0.0
1,427.23,139071,left,2,-87.18215,204.5752,-40.50642,171.17460,247.9453,-96.48488,1.239694,0.685550,1.220096,-137.3886,158.1433,-27.829490,-261.8767,-47.873840,30.62584,0.0,0.000000,0.0
2,427.25,139073,right,2,68.97833,215.1191,-20.12599,-44.44045,302.9170,-83.93305,0.863391,-1.005569,-0.766907,111.5590,167.9522,9.239409,272.0606,-5.603438,83.23297,0.0,0.090605,0.0
3,427.25,139073,left,2,-85.20093,209.7387,-41.18375,70.93041,344.8777,-46.40427,1.294470,0.710267,1.276068,-135.5462,162.8801,-30.982930,-259.9806,-43.469070,26.40578,0.0,0.000000,0.0
4,427.26,139075,right,2,68.11068,220.9579,-21.77186,-22.08052,307.0574,-67.08289,0.875840,-1.017380,-0.767062,110.1764,173.1205,7.244619,267.7276,-2.693964,82.23973,0.0,0.069295,0.0


In [32]:
# sync.time = sync.time.agg(lambda x: x-427)

In [33]:
# sync.set_index("Time", drop=True, inplace=False)

In [34]:
left = sync.groupby("hand_type").get_group("left").drop(["hand_type", "n_hands"], axis=1)
right = sync.groupby("hand_type").get_group("right").drop(["hand_type", "n_hands"], axis=1)

In [35]:
keep_same = {"time", "frame_id"}
left.columns = left.columns.map(lambda x: x if x in keep_same else x + "_left")
right.columns = right.columns.map(lambda x: x if x in keep_same else x + "_right")

In [36]:
merged = left.merge(right, how="outer", on=["time", "frame_id"])

#    .drop("frame_id", axis=1)
merged

,time,frame_id,position_x_left,position_y_left,position_z_left,velocity_x_left,velocity_y_left,velocity_z_left,pitch_left,roll_left,yaw_left,wrist_pos_x_left,wrist_pos_y_left,wrist_pos_z_left,elbow_pos_x_left,elbow_pos_y_left,elbow_pos_z_left,grab_strenth_left,grab_angle_left,pinch_strength_left,position_x_right,position_y_right,position_z_right,velocity_x_right,velocity_y_right,velocity_z_right,pitch_right,roll_right,yaw_right,wrist_pos_x_right,wrist_pos_y_right,wrist_pos_z_right,elbow_pos_x_right,elbow_pos_y_right,elbow_pos_z_right,grab_strenth_right,grab_angle_right,pinch_strength_right
0,427.23,139071,-87.18215,204.5752,-40.50642,171.174600,247.94530,-96.48488,1.239694,0.685550,1.220096,-137.3886,158.1433,-27.82949,-261.8767,-47.87384,30.62584,0.0,0.0,0.0,69.65445,209.9525,-18.54797,-65.047080,338.263100,-102.57630,0.865190,-1.009409,-0.763543,112.1363,162.7645,10.928120,275.1779,-7.356547,87.30630,0.0,0.059833,0.0
1,427.25,139073,-85.20093,209.7387,-41.18375,70.930410,344.87770,-46.40427,1.294470,0.710267,1.276068,-135.5462,162.8801,-30.98293,-259.9806,-43.46907,26.40578,0.0,0.0,0.0,68.97833,215.1191,-20.12599,-44.440450,302.917000,-83.93305,0.863391,-1.005569,-0.766907,111.5590,167.9522,9.239409,272.0606,-5.603438,83.23297,0.0,0.090605,0.0
2,427.26,139075,-83.49533,214.9115,-42.59961,85.710340,348.23460,-102.02660,1.331296,0.730335,1.312464,-133.7321,167.7086,-33.75665,-257.4434,-37.24958,29.84401,0.0,0.0,0.0,68.11068,220.9579,-21.77186,-22.080520,307.057400,-67.08289,0.875840,-1.017380,-0.767062,110.1764,173.1205,7.244619,267.7276,-2.693964,82.23973,0.0,0.069295,0.0
3,427.28,139077,-83.51068,221.4817,-44.18541,0.287095,388.70420,-86.57835,1.358686,0.747044,1.338415,-133.4904,173.9064,-36.19991,-255.7929,-29.82336,33.75477,0.0,0.0,0.0,68.26217,226.7594,-22.93307,7.860982,335.314400,-61.81393,0.883429,-1.023885,-0.767406,110.0817,178.5817,5.874156,267.2276,2.777664,81.73924,0.0,0.058568,0.0
4,427.30,139078,-83.72626,224.9673,-44.57369,-24.301780,392.91710,-43.76951,1.375643,0.755893,1.355570,-133.5924,177.2040,-37.22458,-254.6681,-26.72015,34.28068,0.0,0.0,0.0,68.29678,230.0778,-23.70698,3.900916,374.071900,-87.24055,0.890813,-1.033500,-0.761958,109.6549,181.4484,5.007799,266.7950,5.949927,81.58895,0.0,0.059980,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4521,502.13,147698,-94.49778,257.4222,55.64758,36.686230,-78.04737,-178.26260,1.323389,1.258593,1.065970,-127.3144,194.4717,59.52803,-299.5664,15.22464,16.52840,0.0,0.0,0.0,111.39700,182.9246,34.27466,-42.475920,-117.845800,-61.32522,0.395246,-1.971680,-0.640369,154.4715,151.6662,81.147250,367.3650,47.177640,167.21540,0.0,0.176508,0.0
4522,502.15,147700,-94.05425,255.8155,52.81230,30.153270,-144.97720,-155.62700,1.285448,1.277849,1.007618,-126.9333,193.0801,58.90355,-298.9101,12.87815,18.89758,0.0,0.0,0.0,110.08700,180.8389,32.92039,-71.148420,-111.439600,-69.28531,0.365014,-1.987410,-0.661939,154.3424,151.2262,79.876530,364.2306,45.544070,171.67780,0.0,0.147999,0.0
4523,502.17,147702,-93.68062,253.3679,50.37835,32.269880,-161.19530,-137.36660,1.246310,1.297078,0.956325,-126.9241,191.1363,58.91284,-302.6545,12.99574,26.75648,0.0,0.0,0.0,109.17140,179.3461,31.84416,-46.123690,-69.914710,-57.69822,0.355285,-1.995915,-0.671905,153.8897,150.4046,78.829510,362.5193,46.762530,175.68630,0.0,0.137555,0.0
4524,502.18,147704,-93.12487,250.4072,48.05837,32.141470,-157.05030,-124.67950,1.208146,1.327065,0.901075,-126.4018,188.6597,59.13661,-308.9988,15.97083,37.08796,0.0,0.0,0.0,108.90170,178.8129,31.02029,-15.531000,-5.445981,-50.80544,0.355584,-2.007045,-0.671661,153.5864,150.0203,78.108750,363.4342,50.553340,176.69620,0.0,0.144381,0.0


In [80]:
# adds a column with the second
merged["round_time"] = merged.time.astype(int) - merged.time.min().astype(int)

In [37]:
# reset index (drop = False) adds round time as the left column in the DataFrame
merged = merged.groupby("round_time").nth([0, 1, 2, 3]).reset_index()

KeyError: 'round_time'

In [ ]:
# drop is True to prevent from adding the old indices as a new column
merged = merged.drop(merged[merged["round_time"] < merged.round_time.min() + 7].index).reset_index(drop=True)

In [ ]:
merged

In [86]:
round_time_groups = merged.groupby("round_time")

In [42]:
X = pd.DataFrame(round_time_groups.get_group(441).unstack()).T.append(
    pd.DataFrame(round_time_groups.get_group(442).reset_index(drop=True).unstack()).T)
X

elbow_pos_x_left                                                             \
                0         1        2         3         28        29        30   
0              NaN       NaN      NaN       NaN -318.1297 -318.6808 -319.1055   
0        -277.1588 -281.9228 -288.142 -290.7782       NaN       NaN       NaN   

            elbow_pos_x_right                                                 \
         31                0         1       2        3         28        29   
0 -319.3068               NaN       NaN     NaN      NaN  349.6196  345.2472   
0       NaN          342.8213  356.3757  365.07  368.133       NaN       NaN   

                      elbow_pos_y_left                                \
         30        31               0         1         2         3    
0  338.5471  330.7361              NaN       NaN       NaN       NaN   
0       NaN       NaN         -32.4144 -32.64224 -31.89404 -31.72424   

                                          elbow_pos_y_right            \
         28        29        30        31                0         1    
0 -51.25742 -48.80588 -46.20153 -42.04137               NaN       NaN   
0       NaN       NaN       NaN       NaN         -19.87595 -14.13137   

                                                               \
         2         3         28        29        30        31   
0       NaN       NaN -21.77575 -19.43496 -17.59132 -16.85076   
0 -13.07153 -14.17725       NaN       NaN       NaN       NaN   

  elbow_pos_z_left                                                    \
                0         1         2         3         28        29   
0              NaN       NaN       NaN       NaN  34.35236  30.59289   
0         46.54707  44.98996  44.39068  44.22089       NaN       NaN   

                      elbow_pos_z_right                               \
         30        31                0        1         2         3    
0  27.95444  27.97145               NaN      NaN       NaN       NaN   
0       NaN       NaN          64.49064  66.3457  68.40325  68.86563   

                                          grab_angle_left                      \
         28        29        30        31              0    1    2    3    28   
0  130.6426  126.1261  124.0072  121.3898             NaN  NaN  NaN  NaN  0.0   
0       NaN       NaN       NaN       NaN             0.0  0.0  0.0  0.0  NaN   

                 grab_angle_right                                     \
    29   30   31               0    1    2    3    28   29   30   31   
0  0.0  0.0  0.0              NaN  NaN  NaN  NaN  0.0  0.0  0.0  0.0   
0  NaN  NaN  NaN              0.0  0.0  0.0  0.0  NaN  NaN  NaN  NaN   

  grab_strenth_left                                    grab_strenth_right  \
                 0    1    2    3    28   29   30   31                 0    
0               NaN  NaN  NaN  NaN  0.0  0.0  0.0  0.0                NaN   
0               0.0  0.0  0.0  0.0  NaN  NaN  NaN  NaN                0.0   

                                     pinch_strength_left                      \
    1    2    3    28   29   30   31                  0    1    2    3    28   
0  NaN  NaN  NaN  0.0  0.0  0.0  0.0                 NaN  NaN  NaN  NaN  0.0   
0  0.0  0.0  0.0  NaN  NaN  NaN  NaN                 0.0  0.0  0.0  0.0  NaN   

                 pinch_strength_right                                     \
    29   30   31                   0    1    2    3    28   29   30   31   
0  0.0  0.0  0.0                  NaN  NaN  NaN  NaN  0.0  0.0  0.0  0.0   
0  NaN  NaN  NaN                  0.0  0.0  0.0  0.0  NaN  NaN  NaN  NaN   

  pitch_left                                                            \
          0         1        2        3         28        29        30   
0        NaN       NaN      NaN      NaN  1.625889  1.623542  1.620962   
0   1.460463  1.464737  1.46526  1.46547       NaN       NaN       NaN   

            pitch_right                                                    \
         31 

### Notes:

01:37:00

- Organize the data in a pandas dataframe
- Goal: using the hand ... , detect if is alone, sync or spontaneous.
- "spontaneous synchronizing"
- interpersonal space
- training has 9 participants
- validation has different participants
- spontan and sync: if # hands is 1, remove data!
- time series
- every 2 lines is one feature (need to be combined) = 1 frame
- choose how many frames
- position y is similar
- 2 recordings, second is usually better
- can't split train and test as usual (correlation between each following movement). can't shuffle!
for instance, can take first 40 seconds for training and last 5 for testing.
- at least 2 seconds in between train and test
- model for 1 person, try model on second person. train the second person and test the third etc.
- 4 frames per second
-

> #### Resources:
1. Leap Motion Attributes <a href="https://developer-archive.leapmotion.com/documentation/python/api/Leap.Hand.html"
> title="leapmotion">link</a>
